In [ ]:
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from tqdm import tqdm
import pickle
import random
import numpy as np
from collections import Counter, defaultdict
import numpy as np
from torch import FloatTensor as FT

### Instructions
For this part, fill in the required code and make the notebook work. This wll be very similar to the Skip-Gram model, but a little more difficult. Look for the """ FILL IN """ string to guide you.

In [ ]:
# Where do I want to run my job. You can do "cuda" on linux machines
DEVICE = "mps" if torch.backends.mps.is_available() else  "cpu"
# DEVICE = "cuda" if torch.cuda.is_available() else  "cpu"

# The batch size in Adam or SGD
BATCH_SIZE = 512

# Number of epochs
NUM_EPOCHS = 10

# Predict from 2 words the inner word for CBOW
# I.e. I'll have a window like ["a", "b", "c"] of continuous text (each is a word)
# We'll predict each of wc = ["a", "c"] from "b" = wc for Skip-Gram
# For CBOW, we'll use ["a", "c"] to predict "b" = wo
WINDOW = 1

# Negative samples.
K = 4

The text8 Wikipedia corpus. 100M characters.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!du -h text8

f = open('/content/drive/MyDrive/text8/text8', 'r')
text = f.read()
# One big string of size 100M
print(len(text))

100000000


In [ ]:
punc = '!"#$%&()*+,-./:;<=>?@[\\]^_\'{|}~\t\n'

# Can do regular expressions here too
for c in punc:
    if c in text:
        text.replace(c, ' ')

In [ ]:
# A very crude tokenizer you get for free: lower case and also split on spaces
# This will not work!
# Split text on space and strip each word
# You should get a list "words" which is text but each element is a word in order
TOKENIZER = get_tokenizer("basic_english")
""" FILL IN """

In [ ]:
words = TOKENIZER(text)
f = Counter(words)
# Fix the above
""" FILL IN """

In [ ]:
len(words)

17005207

In [ ]:
# Do a very crude filter on the text which removes all very popular words
text = [word for word in words if f[word] > 5]

In [ ]:
text[0:5]

['anarchism', 'originated', 'as', 'a', 'term']

In [ ]:
VOCAB = build_vocab_from_iterator([text])
# Rebuild the vocabulary from text above
""" FILL IN """

In [ ]:
# Populate these maps manually using Counter or defaltdict
# This will not work
""" FILL IN """

# word -> int hash map
stoi = VOCAB.get_stoi()
# int -> word hash map
itos = VOCAB.get_itos()

In [ ]:
stoi['as']

11

In [ ]:
# Total number of words; you should see about 63K as below
len(stoi)

63641

In [ ]:
f = Counter(text)
# This is the probability that we pick a word in the corpus
z = {word: f[word] / len(text) for word in f}

In [ ]:
threshold = 1e-5
# Probability that word is kept while subsampling
# This is explained here and sightly differet from the paper: http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
p_keep = {word: (np.sqrt(z[word] / 0.001) + 1)*(0.0001 / z[word]) for word in f}

In [ ]:
# This is in the integer space
train_dataset = [word for word in text if random.random() < p_keep[word]]

# Rebuild the vocabulary
VOCAB = build_vocab_from_iterator([train_dataset])

In [ ]:
len(train_dataset)

7847755

In [ ]:
# Repopulate the stoi and itos maps again now that you dropped some words
""" FILL IN """
# word -> int mapping
stoi = VOCAB.get_stoi()
# int -> word mapping
itos = VOCAB.get_itos()

In [ ]:
# The vocabulary size after we do all the filters
len(VOCAB)

63641

In [ ]:
# The probability we draw something for negative sampling
f = Counter(train_dataset)
p = torch.zeros(len(VOCAB))

# Downsample frequent words and upsample less frequent
s = sum([np.power(freq, 0.75) for word, freq in f.items()])

for word in f:
    p[stoi[word]] = np.power(f[word], 0.75) / s

In [ ]:
# Map everything to integers
# This might not work be careful w the above ...
train_dataset = [stoi[word] for word in text]

In [ ]:
# This just gets the (wc, wo) pairs that are positive - they are seen together!
def get_tokenized_dataset(dataset, verbose=False):
    x_list = []

    for i, token in enumerate(dataset):
        m = 1

        # Get the left and right tokens
        start = """ FILL IN """
        left_tokens = """ FILL IN """

        end = """ FILL IN """
        right_tokens = """ FILL IN """

        # Check these are the same length, and if so use them to add a row of data. This should be a list like
        # [a, c, b] where b is the center word
        if len(left_tokens) == len(right_tokens):
            w_context = """ FILL IN """

            wc = """ FILL IN """

            x_list.extend(
                """ FILL IN """
            )

    return x_list

In [ ]:
train_x_list = get_tokenized_dataset(train_dataset, verbose=False)

In [ ]:
pickle.dump(train_x_list, open('train_x_list.pkl', 'wb'))

In [ ]:
train_x_list = pickle.load(open('train_x_list.pkl', 'rb'))

In [ ]:
# These are (wc, wo) pairs. All are y = +1 by design
train_x_list[:10]

[[5233, 11, 3083],
 [3083, 6, 11],
 [11, 190, 6],
 [6, 1, 190],
 [190, 3133, 1],
 [1, 45, 3133],
 [3133, 59, 45],
 [45, 138, 59],
 [59, 134, 138],
 [138, 741, 134]]

In [ ]:
# The number of things of BATCH_SIZE = 512
assert(len(train_x_list) // BATCH_SIZE == 32579)

32579

### Set up the dataloader.

In [ ]:
train_dl = DataLoader(
    TensorDataset(
        torch.tensor(train_x_list).to(DEVICE),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [ ]:
for xb in train_dl:
    assert(xb[0].shape == (BATCH_SIZE, 3))
    break

### Words we'll use to asses the quality of the model ...

In [ ]:
valid_ids = torch.tensor([
    stoi['money'],
    stoi['lion'],
    stoi['africa'],
    stoi['musician'],
    stoi['dance'],
])

### Get the model.

In [ ]:
class CBOWNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CBOWNegativeSampling, self).__init__()
        self.A = """ FILL IN """ # Context vectors - center word
        self.B = """ FILL IN """ # Output vectors - words around the center word
        self.init_weights()

    def init_weights(self):
        # Is this the best way? Not sure
        initrange = 0.5
        self.A.weight.data.uniform_(-initrange, initrange)
        self.B.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        # N is the batch size
        # x is (N, 3)

        # Context words are 2m things, m = 1 so w_context is (N, 2) while wc is (N, 1)
        w_context, wc = """ FILL IN """

        # Each of these is (N, 2, D) since each context has 2 word
        # We want this to be (N, D) and this is what we get

        # (N, 2, D)
        a = """ FILL IN """

        # (N, D)
        a_avg = """ FILL IN """

        # Each of these is (N, D) since each target has 1 word
        b = """ FILL IN """

        # The product between each context and target vector. Look at the Skip-Gram code.
        # The logits is now (N, 1) since we sum across the final dimension.
        logits = """ FILL IN """

        return logits

In [ ]:
@torch.no_grad()
def validate_embeddings(
    model,
    valid_ids,
    itos
):
    """ Validation logic """

    # We will use context embeddings to get the most similar words
    # Other strategies include: using target embeddings, mean embeddings after avaraging context/target
    embedding_weights = model.A.weight

    normalized_embeddings = embedding_weights.cpu() / np.sqrt(
        np.sum(embedding_weights.cpu().numpy()**2, axis=1, keepdims=True)
    )

    # Get the embeddings corresponding to valid_term_ids
    valid_embeddings = normalized_embeddings[valid_ids, :]

    # Compute the similarity between valid_term_ids (S) and all the embeddings (V)
    # We do S x d (d x V) => S x D and sort by negative similarity
    top_k = 10 # Top k items will be displayed
    similarity = np.dot(valid_embeddings.cpu().numpy(), normalized_embeddings.cpu().numpy().T)

    # Invert similarity matrix to negative
    # Ignore the first one because that would be the same word as the probe word
    similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]

    # Print the output.
    for i, word_id in enumerate(valid_ids):
        # j >= 1 here since we don't want to include the word itself.
        similar_word_str = ', '.join([itos[j] for j in similarity_top_k[i, :] if j >= 1])
        # This might need a fix!
        print(f"{itos[word_id]}: {similar_word_str}")

    print('\n')

### Set up the model

In [ ]:
LR = 10.0
NUM_EPOCHS = 10
EMBED_DIM = 300

In [ ]:
model = CBOWNegativeSampling(len(VOCAB), EMBED_DIM).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

# The learning rate is lowered every epoch by 1/10
# Is this a good idea?
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)

In [ ]:
model

In [ ]:
validate_embeddings(model, valid_ids, itos)

### Train the model

In [ ]:
ratios = []

def train(dataloader, model, optimizer, epoch):
    model.train()
    total_acc, total_count, total_loss, total_batches = 0, 0, 0.0, 0.0
    log_interval = 500

    for idx, x_batch in tqdm(enumerate(dataloader)):

        x_batch = x_batch[0]

        batch_size = x_batch.shape[0]

        # Zero the gradient so they don't accumulate
        """ FILL IN """

        logits = model(x_batch)

        # Get the positive samples loss. Notice we use weights here
        positive_loss = """ FILL IN """

        # For each batch, get some negative samples
        # We need a total of len(y_batch) * K samples across a batch
        # We then reshape this batch
        # These are effectively the output words
        negative_samples = """ FILL IN """

        # Context words are 2m things, m = 1 so w_context is (N, 2) while wc is (N, 1)
        w_context, wc = """ FILL IN """

        """
        if w_context looks like below (batch_size = 3)
        [
        (a, b),
        (c, d),
        (e, f)
        ] and K = 2 we'd like to get:

        [
        (a, b),
        (a, b),
        (c, d),
        (c, d),
        (e, f),
        (e, f)
        ]

        This will be batch_size * K rows.
        """

        # This should be (N * K, 2)
        w_context = torch.concat([
            w.repeat(K, 1) for w in torch.tensor(w_context).split(1)
        ])

        # Remove the last dimension 1
        wc = """ FILL IN """

        # Get the negative samples. This should be (N * K, 3)
        # Concatenate the w_context and wc along the column. Make sure everything is on CUDA / MPS or CPU
        x_batch_negative = """ FILL IN """

        """
        Note the way we formulated the targets: they are all 0 since these are negative samples.
        We do the BCEWithLogitsLoss by hand basically here.
        Notice we sum across the negative samples, per positive word.

        This is literally the equation in the lecture notes.
        """

        # (N, K, D) -> (N, D) -> (N)
        # Look at the Skip-Gram notebook
        negative_loss = """ FILL IN """

        loss = """ FILL IN """

        # Get the gradients via back propagation
        """ FILL IN """

        # Clip the gradients? Generally a good idea
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)


        # Do an optimization step. Update the parameters A and B
        """ FILL IN """

        # Get the new loss
        total_loss += """ FILL IN """

        # Update the batch count
        total_batches += """ FILL IN """

        if idx % log_interval == 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| loss {:8.3f} ".format(
                    epoch,
                    idx,
                    len(dataloader),
                    total_loss / total_batches
                )
            )
            validate_embeddings(model, valid_ids, itos)
            total_loss, total_batches = 0.0, 0.0

### Some results from the run look like below:

Somewhere inside of 2 iterations you should get sensible associattions.
Paste here a screenshot of the closest vectors.

In [ ]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()

    train(train_dl, model, optimizer, epoch)
    # We have a learning rate scheduler here

    # Basically, given the state of the optimizer, this lowers the learning rate in a smart way
    scheduler.step()

0it [00:00, ?it/s]/var/folders/x8/2_vxppc52znb82mg86nv4y000000gn/T/ipykernel_9301/3264625997.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w.repeat(K, 1) for w in torch.tensor(w_context).split(1)
1it [00:00,  2.46it/s]

| epoch   1 |     0/32580 batches | loss    4.027 
money: coyotes, furtado, flanking, managed, infogrames, startrek, reyna, damian, larry, pseudonyms
lion: aksai, dumpster, interpolation, mixture, rale, sabor, aphasia, prokaryotes, gametophyte, urology
africa: virginian, wendish, caryophyllales, electricity, warlike, swamp, bombay, catechisms, surrealistic, jacques
musician: lothal, exponent, orsay, awami, screwed, aedui, vassalage, smalltalk, padova, ivo
dance: akhmatova, monty, chunk, bofors, desqview, nephew, emitter, quaternary, recombination, strasser




501it [01:11,  6.33it/s]

| epoch   1 |   500/32580 batches | loss    3.719 
money: coyotes, furtado, flanking, managed, infogrames, startrek, reyna, damian, larry, speedway
lion: aksai, dumpster, interpolation, mixture, rale, sabor, aphasia, prokaryotes, gametophyte, urology
africa: virginian, wendish, caryophyllales, bombay, jacques, electricity, warlike, inactivation, surrealistic, catechisms
musician: lothal, exponent, orsay, awami, screwed, vassalage, aedui, smalltalk, padova, gatt
dance: akhmatova, monty, bofors, chunk, desqview, nephew, emitter, quaternary, recombination, strasser




1001it [02:21,  6.29it/s]

...

27501it [1:05:08,  6.36it/s]

| epoch   1 | 27500/32580 batches | loss    1.453 
money: support, work, god, death, example, use, production, end, view, much
lion: met, mixture, introduced, abuse, included, discussed, question, saints, society, pope
africa: america, end, north, europe, india, china, nine, england, result, united
musician: actress, actor, singer, writer, four, composer, north, nine, eight, england
dance: terms, range, united, other, list, country, zero, end, actor, york




28001it [1:06:18,  6.35it/s]

| epoch   1 | 28000/32580 batches | loss    1.457 
money: support, work, use, god, death, example, production, view, end, nine
lion: met, mixture, introduced, abuse, included, discussed, question, saints, pope, society
africa: america, end, north, europe, nine, india, china, england, result, united
musician: actress, actor, singer, nine, writer, composer, north, rest, eight, four
dance: terms, united, range, other, list, country, actor, end, york, nine




28451it [1:07:21,  7.09it/s]